In [7]:
import numpy as np
import matplotlib.pyplot as plt
from math import log
from scipy.special import erf

%matplotlib inline

In [2]:
n = np.arange(10, 100 + 10, 10)
p = np.arange(0.5, 1 + 0.01, 0.01)

def empirical_dist(_data):
    unique_data = np.unique(_data)
    _dist = np.zeros_like(unique_data)
    for i in range(unique_data.shape[0]):
        _dist[i] = np.sum(_data < unique_data[i]) / _data.shape[0]
    return _dist



In [422]:
from scipy.stats import chi2
def laplace_func(x):
   return  erf(x / 2 ** 0.5) / 2

def pearsons_test(distribution, significance_level):
    n = distribution.shape[0]
    m = np.mean(distribution)
    sd = np.sqrt(np.sum((distribution - m) ** 2) / (n - 1))
    dist_interval = np.max(distribution) - np.min(distribution)
    del_x = dist_interval / (1 + 3.3 * (log(n)))
    borders = np.arange(np.min(distribution), np.max(distribution) + del_x, del_x)
    n_i = np.array([np.count_nonzero((distribution >= borders[i]) & (distribution < borders[i+1] )) \
        for i in range(borders.shape[0] - 1)])
    z = np.array([((i - m) / sd) for i in borders])
    p_i = np.array([ (laplace_func(z[i + 1]) - laplace_func(z[i])) for i in range(z.shape[0] - 1) ])
    chi_sqare = np.sum( ((n_i - n * p_i) ** 2) / (n * p_i) )
    k = n_i.shape[0] - 2 - 1 # For normal distribution we have only 2 degrees of freedom
    chi_critical = chi2.isf(significance_level, k)
    return {"Chi_square" : chi_sqare, "Chi_critical" : chi_critical}

def jarque_bera_test (distribution):
    n = distribution.shape[0]
    m = np.mean(distribution)
    sd = np.sqrt(np.sum((distribution - m) ** 2) / (n - 1))
    skewness = np.sum((distribution - m) ** 3) / (n * sd ** 3)
    kurtosis = (np.sum((distribution - m) ** 4) / (n * sd ** 4)) - 3
    return n/6 *(skewness**2 + kurtosis**2 / 4)

data = np.loadtxt('./data.txt', delimiter=' ')
print(pearsons_test(data, 0.05))
print(jarque_bera_test(data))

{'Chi_square': 15.408567412395325, 'Chi_critical': 22.362032494826945}
2.625473691973074


In [2]:
from numpy import e, inf
from scipy.special import gamma, gammainc, gammaincinv
from scipy.integrate import quad
from decimal import Decimal
from scipy.misc import derivative

def lower_incomplete_gamma_function(s: float = .0, x: float = .0):
    return (quad(lambda t: (t ** (s - 1)) * (e ** (-t)), 0, x)[0])


def gamma_function(z: float = .0):
    return (quad(lambda t: (t ** (z - 1)) * (e ** (-t)), 0, inf)[0])

def F(x, k):
    li = (lower_incomplete_gamma_function(k / 2, x / 2))
    g = (gamma_function(k / 2))
    print("gamma_incomplete", li, gammainc(k / 2, x / 2))
    print("gamma", g, gamma(k / 2))
    #print(1 - li / g)
    res = li / g
    #res = gammainc(k / 2, x / 2) / gamma(k / 2)
    #res = np.float128(5.8115079115215259e-12) / np.float128(287.88527781504445)
    return  res, li, g

def Finv(x, k):
    f = F(x, k)
    print(f[0])
    res = 2 * gammaincinv(k / 2, 1 - x) #(1 - gammaincinv(f, x / 2)) * gamma(k / 2)
    #res = 

    return res

print(Finv(0.05, 13))

gamma_incomplete 5.811507911521526e-12 2.0186888178613977e-14
gamma 287.88527781504445 287.88527781504433
2.0186888178613992e-14
22.362032494826934
